# NLPeasy Demonstration

In [1]:
import pandas as pd
import nlpeasy as ne

Connect to running elastic or else start an Open Source stack on your docker

In [2]:
elk = ne.connect_elastic(dockerPrefix='nlp', elkVersion='7.4.0', mountVolumePrefix=None)

'No elasticsearch on localhost:9200 found, trying connect to docker container with prefix nlp'

'No docker container with prefix nlp; starting one'

ElasticSearch on http://localhost:32770
Kibana on http://localhost:32771

> If it is started on docker it will on the first time pull the images (1.3GB)!
BTW, this function is not blocking, i.e. the servers might only be active couple of seconds later.
Setting mountVolumePrefix="./elastic-data/" would keep the data of elastic in your
filesystems and then the data survives container restarts

read data as Pandas data frame

In [3]:
nips = pd.read_pickle("data_raw/nips.pickle")

setup stages in the NLP pipeline and set textfields

In [4]:
pipeline = ne.Pipeline(index='nips', textCols=['message','title'], dateCol='year', elk=elk)

pipeline += ne.RegexTag(r'\$([^$]+)\$', ['message'], 'math')
pipeline += ne.VaderSentiment('message', 'sentiment')
pipeline += ne.SpacyEnrichment(nlp='en_core_web_md', cols=['message','title'])

Run the pipeline

In [5]:
nips_enriched = pipeline.process(nips, writeElastic=True)

8250/8250 [==============================] - 5:12 38ms/step


Create Kibana Dashboard of all the columns

In [6]:
pipeline.create_kibana_dashboard()

nips: adding index-pattern
nips: setting default index-pattern
nips: adding search
nips: adding visualisation for year
nips: adding visualisation for math
nips: adding visualisation for message_ents
nips: adding visualisation for message_subj
nips: adding visualisation for message_verb
nips: adding visualisation for title_ents
nips: adding visualisation for title_subj
nips: adding visualisation for title_verb
nips: adding visualisation for message
nips: adding visualisation for title
nips: adding visualisation for sentiment
nips: adding dashboard
nips: setting time defaults


Open Kibana in webbrowser

In [7]:
elk.show_kibana()

# Shutdown

If elastic was started on Docker and you want to shutdown the servers issue:

In [8]:
# ne.docker.stop_elastic_on_docker("nlp")

> *Warning*: If you didn't use a mountVolumePrefix when you started the servers, all the data in elastic and kibana will be lost!